###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el 	Departamento Nacional de Planeación

In [1]:
import pandas as pd

###Archivo nkjx-rsq7.csv

Contiene información sobre la medición de desempeño municipal, que tiene como objetivo medir y comparar el desempeño municipal entendido como la gestión de las Entidades Territoriales y la consecución de resultados de desarrollo (el aumento de la calidad de vida de la población)




In [2]:
df = pd.read_csv("https://www.datos.gov.co/resource/nkjx-rsq7.csv?$limit=500000", dtype={'codigo_entidad': str})
df.head()

,codigo_unico,indicador,codigo_categoria,codigo_subcategoria,codigo_variable,codigo_departamento,departamento,codigo_entidad,entidad,dato,etiqueta,anio,mes
0,80010001,Grupo de dotaciones iniciales,8,1,1,5,Antioquia,05001,Medellín,NaN,Ciudades,2016,12
1,80010001,Grupo de dotaciones iniciales,8,1,1,5,Antioquia,05002,Abejorral,NaN,G3- Nivel Medio,2016,12
2,80010001,Grupo de dotaciones iniciales,8,1,1,5,Antioquia,05004,Abriaquí,NaN,G3- Nivel Medio,2016,12
3,80010001,Grupo de dotaciones iniciales,8,1,1,5,Antioquia,05021,Alejandría,NaN,G2- Nivel Medio Alto,2016,12
4,80010001,Grupo de dotaciones iniciales,8,1,1,5,Antioquia,05030,Amagá,NaN,G2- Nivel Medio Alto,2016,12


##Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22020 entries, 0 to 22019
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   codigo_unico         22020 non-null  int64  
 1   indicador            22020 non-null  object 
 2   codigo_categoria     22020 non-null  int64  
 3   codigo_subcategoria  22020 non-null  int64  
 4   codigo_variable      22020 non-null  int64  
 5   codigo_departamento  22020 non-null  int64  
 6   departamento         22000 non-null  object 
 7   codigo_entidad       22020 non-null  object 
 8   entidad              22020 non-null  object 
 9   dato                 16515 non-null  float64
 10  etiqueta             5505 non-null   object 
 11  anio                 22020 non-null  int64  
 12  mes                  22020 non-null  int64  
dtypes: float64(1), int64(7), object(5)
memory usage: 2.2+ MB


### Eliminación de columnas irrelevantes para el proyecto

In [17]:
relevant_cols = ['indicador', 'departamento', 'codigo_entidad', 'entidad', 'dato', 'etiqueta', 'anio']
municipal_perfomance_measure_dnp = df[relevant_cols]
municipal_perfomance_measure_dnp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22020 entries, 0 to 22019
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   indicador       22020 non-null  object 
 1   departamento    22000 non-null  object 
 2   codigo_entidad  22020 non-null  object 
 3   entidad         22020 non-null  object 
 4   dato            16515 non-null  float64
 5   etiqueta        5505 non-null   object 
 6   anio            22020 non-null  int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 1.2+ MB


- Verificar periodos

In [18]:
municipal_perfomance_measure_dnp['anio'].unique()

array([2016, 2017, 2018, 2019, 2020])

### Verificar valores nulos

In [19]:
municipal_perfomance_measure_dnp.isnull().sum()

,0
indicador,0
departamento,20
codigo_entidad,0
entidad,0
dato,5505
etiqueta,16515
anio,0


### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [23]:
# Imprimir categorías únicas para columnas de tipo object
categorical_col = ['indicador', 'departamento', 'entidad', 'etiqueta']
for column in categorical_col:
    print(f"Categorías en la columna '{column}':")
    print(municipal_perfomance_measure_dnp[column].unique())
    print()

Categorías en la columna 'indicador':
['GRUPO DE DOTACIONES INICIALES' 'COMPONENTE DE GESTION'
 'COMPONENTE DE RESULTADOS' 'MDM']

Categorías en la columna 'departamento':
['Antioquia' 'Atlántico' nan 'Bolívar' 'Boyacá' 'Caldas' 'Caquetá' 'Cauca'
 'Cesar' 'Córdoba' 'Cundinamarca' 'Chocó' 'Huila' 'La Guajira' 'Magdalena'
 'Meta' 'Nariño' 'Norte de Santander' 'Quindío' 'Risaralda' 'Santander'
 'Sucre' 'Tolima' 'Valle del Cauca' 'Arauca' 'Casanare' 'Putumayo'
 'San Andrés y Providencia' 'Amazonas' 'Guainía' 'Guaviare' 'Vaupés'
 'Vichada']

Categorías en la columna 'entidad':
['Medellín' 'Abejorral' 'Abriaquí' ... 'La Primavera' 'Santa Rosalía'
 'Cumaribo']

Categorías en la columna 'etiqueta':
['Ciudades' 'G3- Nivel Medio' 'G2- Nivel Medio Alto' 'G1- Nivel Alto'
 'G4- Nivel Medio Bajo' 'G5- Nivel Bajo' nan]



- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [33]:
import unicodedata

def remove_accents_and_special_chars(input_str):
  if isinstance(input_str, str):  # Solo procesar si el valor es una cadena
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars = [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result
  else:
    return input_str

In [34]:
# Aplicar la función a todas las columnas categóricas
for col in categorical_col:
    municipal_perfomance_measure_dnp.loc[:, col] = municipal_perfomance_measure_dnp[col].apply(remove_accents_and_special_chars)

- Codificación de algunas variables categóricas

### Verificar que los valores en "codigo_entidad" coincidan con los códigos reales de municipios

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres.

Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigo_mpio"

In [36]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


 -  Verificar la consistencia de la columna "codigo_entidad" en el df municipal_perfomance_measure_dnp

In [37]:
# Asegurarnos de que todos los valores en 'codigo_m' sean strings
municipal_perfomance_measure_dnp.loc[:,'codigo_entidad'] = municipal_perfomance_measure_dnp['codigo_entidad'].astype(str)

# Calcular la longitud de cada valor en la columna
longitudes = municipal_perfomance_measure_dnp['codigo_entidad'].apply(len)

# Verificar si todas las longitudes son iguales
longitudes.nunique() == 1

True

In [38]:
# Mostrar longitudes únicas (opcional)
print(f"Longitudes únicas: {longitudes.unique()}")

Longitudes únicas: [5]


In [39]:
# Contar registros por longitud
long_df = longitudes.value_counts().reset_index()
long_df.rename(columns={'codigo_dane': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitudes))
long_df.head()

,codigo_entidad,count,percentage
0,5,22020,1.0


In [40]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitudes.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = municipal_perfomance_measure_dnp[longitudes == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigo_entidad']])
    print()

Muestra de registros con longitud 5:
  codigo_entidad
0          05001
1          05002
2          05004
3          05021
4          05030



Nota: el 100% de los cod corresponden a un string de 5 digitos, este es la forma del cod almacenado en la bd

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales

In [41]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [42]:
# Comparar listas de códigos
compare_lists(municipal_perfomance_measure_dnp['codigo_entidad'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en municipal_perfomance_measure_dnp", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en municipal_perfomance_measure_dnp que no están en Códigos de municipios dept_mpios_codes:
set()


### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

In [45]:
# Eliminar columnas que ya no se necesitan
columns_to_drop = ['departamento', 'entidad']
municipal_perfomance_measure_dnp = municipal_perfomance_measure_dnp.drop(columns=columns_to_drop)

In [47]:
# Adicionar columna para trazabilidad de la fuente
municipal_perfomance_measure_dnp['source_id'] = 125

In [48]:
municipal_perfomance_measure_dnp.columns

Index(['indicador', 'codigo_entidad', 'dato', 'etiqueta', 'anio', 'source_id'], dtype='object')

In [49]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'indicador': 'indicator',
    'codigo_entidad': 'dane_code',
    'dato': 'value',
    'etiqueta': 'label',
    'anio': 'year',
    'source_id': 'source_id'
}

# Renombrar las columnas
municipal_perfomance_measure_dnp.rename(columns=translation_map, inplace=True)

In [50]:
#Estructura final del dataset a integrar a la base de datos
municipal_perfomance_measure_dnp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22020 entries, 0 to 22019
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   indicator  22020 non-null  object 
 1   dane_code  22020 non-null  object 
 2   value      16515 non-null  float64
 3   label      5505 non-null   object 
 4   year       22020 non-null  int64  
 5   source_id  22020 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 1.0+ MB


## Salvar en archivo csv en el drive

In [52]:
# Guardar en archivos CSV en el drive
municipal_perfomance_measure_dnp.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/municipal_perfomance_measure_dnp.csv', index=False)